# Lab 8: Grid Localization using Bayes Filter

### <span style="color:rgb(0,150,0)">It is recommended that you close any heavy-duty applications running on your system while working on this lab.</span>

<hr>


# Python Classes
There are five major classes (VirtualRobot, Mapper, BaseLocalization, Plotter and Trajectory)  defined in **robot_interface.py** that provide you with the necessary functions and variables for working your way through the lab. 
In this notebook, we discuss only the relevant functions and variables that are of use for this lab. 
You may refer to the python file for an understanding of how the various functions work, however, this is not entirely necessary as the Jupyter notebook should provide you with all the necessary information.
<hr>

## VirtualRobot
The class provides member functions required to interact with the virtual robot in the simulator. <br> 

**Make sure the simulator is running.** <br>

### Member Functions:
- <span style="color:rgb(255,191,0)">**get_pose()**</span>:  Returns the latest odometry robot pose estimate as a tuple of the format $(x, y, yaw)$ with units $(meters, meters, degrees)$.

- <span style="color:rgb(255,191,0)">**get_gt_pose()**</span>: Returns the latest ground truth robot pose as a tuple of the format $(x, y, yaw)$ with units $(meters, meters, degrees)$.


**<ins>NOTE</ins>**: The above functions provide angle information in degrees.

<hr>

## Mapper
The class provides member functions to handle various operations related to the grid map. <br>
The discrete grid map spans a world of $(-2, 2)~meters$ in the x-axis, $(-2,2)~meters$ in the y-axis and $[-180, 180)~degrees$ in the $\theta$-axis. <br>
The grid cell size in the x, y, and angle axes are 0.2 m, 0.2 m and 20 deg, respectively. <br>
The grid has $(20, 20, 18)$ cells along the different dimensions, with a total number of 7200 cells. <br>   

### Member Functions:
- <span style="color:rgb(255,191,0)">**from_map(cx, cy, ca)**</span>: Returns the continuous world coordinates *(x,y,a)* of the center of the grid cell index *(cx, cy, ca)*. 
- <span style="color:rgb(255,191,0)">**to_map(x, y, a)**</span>: Returns the grid map cell index *(cx, cy, ca)* of the continuous world coordinates *(x,y,a)*.
- <span style="color:rgb(255,191,0)">**normalize_angle(a)**</span>: Normalizes the angle *a* to the interval \[-180, 180).
- <span style="color:rgb(255,191,0)">**get_views(cx, cy, ca)**</span>: Returns a 1D array of the precached true measurements (views) at a given grid cell index *(cx, cy, ca)*. The true measurements are calculated offline by performing ray cast operations from the center of each cell in the map.

### Member Variables:
- <span style="color:rgb(100,191,100)">**MAX_CELLS_X**</span>: Maximum number of grid cells in the X direction.
- <span style="color:rgb(100,191,100)">**MAX_CELLS_Y**</span>: Maximum number of grid cells in the Y direction.
- <span style="color:rgb(100,191,100)">**MAX_CELLS_A**</span>: Maximum number of grid cells in the $\theta$ direction ('A' stands for 'Angle').
- <span style="color:rgb(100,191,100)">**OBS_PER_CELL**</span>: Number of observations per cell. This is set to 18.
- <span style="color:rgb(100,191,100)">**obs_views**</span>: A 4D Numpy array where the first three dimensions represents the cell index (cx, cy, ca) and the fourth dimension represents the precached true measurements (views). Hence the size of the Numpy array is (20,20,18,18). You may think of it as a 3D array representing the grid where each cell stores 18 precached true measurements calculated using ray casting from the center of that cell.

**<ins>NOTE</ins>**: **Do not change the map properties** such as discretization size, map size, maximum number of cells, etc as they are tuned for the specific lab setting.
<hr>

## Plotter
The Plotter class provides various member functions to plot and visualize data. <br>

### Member Functions:

- <span style="color:rgb(255,191,0)">**plot_point(x, y, plot_type)**</span>: Plots a point at the position *x*,*y*. The color of the plotted point depends on *plot_type* which can be equal to one of the following: *ODOM, GT, or BEL* for blue, green or yellow, respectively.

- <span style="color:rgb(255,191,0)">**visualize_map()**</span>: Visualizes the map in the plotter as white lines segments as defined in the member variable **lines** of class **Mapper**.

- <span style="color:rgb(255,191,0)">**visualize_prior_bel()**</span>: Visualizes the prior belief(i.e belief after the prediction step) in the plotter as a grid map where each cell has a grayscale value depending on the probability of the cell; darkers regions indicate lower probability. **Make sure the "Dist." toggle is switched on in the plotter**.

- <span style="color:rgb(255,191,0)">**visualize_bel()**</span>: Visualizes the belief(i.e belief after the update step) in the plotter as a grid map where each cell has a grayscale value depending on the probability of the cell; darkers regions indicate lower probability. **Make sure the "Dist." toggle is switched on in the plotter**.

- <span style="color:rgb(255,191,0)">**reset_plot()**</span>: Resets the plots in the plotter. This removes all the previous plotted information.

- <span style="color:rgb(255,191,0)">**reset_map()**</span>: Clears the map visualization in the plotter.


**<ins>NOTE</ins>**: The functions *visualize_prior_bel()* and *visualize_bel()* visualize a 3D (grid) Belief in a 2D (grid) space by marginalizing over  $\theta$ i.e $bel(x,y) = \sum_{\theta} bel(x,y,\theta)$ and $\overline{bel}(x,y) = \sum_{\theta} \overline{bel}(x,y,\theta)$. <br>
<hr>

## BaseLocalization
The class provides member functions and member variables required for Grid Localization. <br>
Below is a quick summary of the necessary member functions and variables that will be required to complete this lab:

### Member Functions:
- <span style="color:rgb(255,191,0)">**init_pose(x,y,a)**</span>: Initialize the member variable **bel** with a point mass distribution centered at *(x,y,a)*.
- <span style="color:rgb(255,191,0)">**gaussian(x, mu, sigma)**</span>: Returns the relative likelihood of *x* in a Normal Distribution with mean *mu* and standard deviation *sigma*.
- <span style="color:rgb(255,191,0)">**get_observation_data()**</span>: Executes a rotation behavior where the robot rotates in place and collects 18 (=*OBS_PER_CELL*)  range measurements at equidistant angular intervals over a 360 degree space. This mimics a 360 degree laser scanner. The range measurements are stored in the member variable **obs_range_data** of class **BaseLocalization**.
- <span style="color:rgb(255,191,0)">**print_prediction_stats(plot_data)**</span>: A helper function that displays statistics after your prediction step and plots the ground truth position (green), odometry position (blue) and visualizes the prior belief (grayscale grid) if *plot_data* is True.
- <span style="color:rgb(255,191,0)">**print_update_stats(plot_data)**</span>: A helper function that displays statistics after your update step and plots the belief position (yellow) if *plot_data* is True.

### Member Variables:
- <span style="color:rgb(100,191,100)">**obs_range_data**</span>: A 1D Numpy array containing range measurements made by a robot after completing a rotation behaviour i.e after calling the function **get_observation_data()**.
- <span style="color:rgb(100,191,100)">**bel_bar**</span>: A 3D Numpy array that is used to represent the prior belief of the robot i.e after the most recent prediction step.  
- <span style="color:rgb(100,191,100)">**bel**</span>: A 3D Numpy array that is used to represent the belief of the robot i.e after the most recent update step. It is initialized with a point mass distribution centered at $(0~m,0~m,0~deg)$ or grid index $(10,10,9)$.
- <span style="color:rgb(100,191,100)">**sensor_sigma**</span>: The noise parameter (standard deviation of the Gaussian function) for your sensor model.
- <span style="color:rgb(100,191,100)">**odom_rot_sigma**</span>: The noise parameter (standard deviation of the Gaussian function) for rotation in your odometry motion model.
- <span style="color:rgb(100,191,100)">**odom_trans_sigma**</span>: The noise parameter (standard deviation of the Gaussian function) for translation in your odometry motion model.

**<ins>NOTE</ins>**: The noise parameters are dependent on the robot motion model, sensor model and grid discretization. The values are thus tuned to work for the lab setting. **Do not change the noise parameters.**
<hr>

## Trajectory
The trajectory class encodes a pre-planned collision free trajectory to be executed by your robot. The class is defined in **Traj.py**. <br>

### Member Functions:
<span style="color:rgb(255,191,0)">**execute_time_step(t)**</span>: Executes the motion at time step *t* and returns *prev_odom, current_odom, prev_gt, current_gt*.
   - prev_odom: odometry pose before executing the motion.    
   - current_odom: odometry pose after executing the motion.
   - prev_gt     : ground truth pose before executing the motion.
   - current_gt  : ground truth pose after executing the motion.

### Member Variables:
<span style="color:rgb(100,191,100)">**total_time_steps**</span>: The total number of time steps in the trajectory (26). 


Thus, calling the member function **execute_time_step(t)** repeatedly from t=0 to t=total_time_steps-1 will make the robot execute the entire collision-free, pre-planned trajectory.
## <span style="color:rgb(240,50,50)"> Do not use the mouse or the keyboard teleop tool to move the robot while executing the pre-planned trajectory motion. </span>
<hr>

In [1]:
# Automatically reload changes in python modules
%load_ext autoreload
%autoreload 2

# Import classes
from robot_interface import *

import time
import numpy as np
import rospy
from Traj import Trajectory

In [2]:
# Initialize Robot to communicate with the virtual/real robot and plotter
robot = VirtualRobot()

# Initialize mapper
# Requires a VirtualRobot object as input
mapper = init_map(robot)

# Initialize your localization object
# Requires a robot VirtualRobot object and a Mapper object as inputs
loc = BaseLocalization(robot, mapper)

# Visualize the map (described using line segments) in the plotter
loc.plotter.visualize_map()

Using python version: 3.6.9 (default, Jul 17 2020, 12:50:27) 
[GCC 8.4.0] 

Initializing Node
Initializing Virtual Robot
Initializing Mapper
 | Number of observations per grid cell:  18
 | Precaching Views...
 | Precaching Time:  49.0553674697876
Initial Pose:  (10, 10, 9)
Initializing belief with a Point mass Distribution at:  (10, 10, 9)


# Execute Trajectory
The below cell showcases how to use the **Trajectory** class to go through the pre-planned motion trajectory. <br>
The cell does not run any sort of probabilistic estimation. It simply executes the motion in each time step in a successive manner. 

Run this cell once and monitor how the robot executes the entire, collision-free trajectory.

**Make sure you go through the motion in a successive manner in order to execute a collision free trajectory.** 

In [3]:
loc.plotter.reset_plot()

# Intialize the Trajectory object
traj = Trajectory(loc)

# Run through each motion steps
for t in range(0, traj.total_time_steps):
    # Execute control at t=0
    prev_odom, current_odom, prev_gt, current_gt = traj.execute_time_step(t)
    
    # Plot Odom and GT
    loc.plotter.plot_point(current_odom[0], current_odom[1], ODOM)
    loc.plotter.plot_point(current_gt[0], current_gt[1], GT)
    


 | Resetting Robot pose


# Get Observation Data

The robot is designed to executes a 360 rotation behavior where it rotates in place to collect 18 (=*OBS_PER_CELL*) range measurements at equidistant angular intervals over the 360 degree space. This mimics a 360 degree laser scanner. The range measurements are stored in the member variable **obs_range_data** of class **BaseLocalization**.

The cell below runs the **get_observation_data()** to execute this pre-planned behavior once at its current pose. Notice the virtual robot rotating in-place in the simulator.

In [4]:
# Get Observation Data by executing a 360 degree rotation motion
loc.get_observation_data()

# Print the latest observation data stored in the member variable obs_range_data
print(loc.obs_range_data)

 | Executing Observation Loop at: 30 deg/s
[2.2226007  2.65054083 0.60618371 1.854478   1.79465592 1.9349041
 1.5089314  1.99229968 1.85126197 2.20848083 2.4833169  1.30823791
 1.26989758 2.25541162 2.68588686 2.72486186 2.20330954 0.89902109]


The above behavior will be repeated after each time step in the pre-planned trajectory to get the required observation data ($z_{t}$) at each time step.

<hr>

# Localization
<span style="color:rgb(0,150,0)">Make sure you refer to the section "Implementation Tips" in the lab documentation.</span> <br>

The cell below contains skeleton code for various functions required to perform grid localization using Bayes Filter. <br> 
Write necessary code for each function definition. <br>
You may add more functions or edit a function's definition to have a different input (arguments) and/or output (return values). <br>
For example, you may use numpy matrix operations to calculate the sensor model directly in the **update_step()** function and thus skip the *sensor_model()* function.<br>

### <span style="color:rgb(0,150,0)">If you make changes to any of the functions below, make sure to re-run the below cell.</span> <br>


In [5]:
from math import atan2, pi, sqrt, hypot
#In the docstring, "Pose" refers to a tuple (x,y,yaw) in (meters, meters, degrees)
# In world coordinates
def compute_control(cur_pose, prev_pose):
    """ Given the current and previous odometry poses, this function extracts
    the control information based on the odometry motion model.

    Args:
        cur_pose  ([Pose]): Current Pose
        prev_pose ([Pose]): Previous Pose 

    Returns:
        [delta_rot_1]: Rotation 1  (degrees)
        [delta_trans]: Translation (meters)
        [delta_rot_2]: Rotation 2  (degrees)
    """
 
    #this is equal to the amount of translational change in x
    delta_x = cur_pose[0] - prev_pose[0] 
    #this is equal to the amount of translational change in y
    delta_y = cur_pose[1] - prev_pose[1] 
    #this is equal to the amount of rotational change in yaw (converted from radians to degrees)
    #angular change due to a change in position is equal to tan of change in y/change
    delta_turn = atan2(delta_y, delta_x)*180/pi 
    
    #this is equal to the rotational change from the starting angle
    delta_rot_1 = loc.mapper.normalize_angle(delta_turn - prev_pose[2])
    #this is equal to the Euclidean distance between the starting and ending points
    #delta_trans = sqrt((delta_x ** 2) + (delta_y ** 2))
    delta_trans = hypot(delta_x, delta_y)
    #this is equal to the rotational change from the ending angle
    delta_rot_2 = loc.mapper.normalize_angle(cur_pose[2] - delta_turn)

    return delta_rot_1, delta_trans, delta_rot_2
    """
    x_dist = cur_pose[0]-prev_pose[0] # distance traveled in x direction [m]
    y_dist = cur_pose[1]-prev_pose[1] # distance traveled in y direction [m]
    trans_angle = atan2(y_dist, x_dist)*180/pi # angle at which translation happened [deg]
    delta_rot_1 = mapper.normalize_angle(trans_angle-prev_pose[2]) # rotaion from start pose to translation angle
    delta_trans = hypot(x_dist, y_dist) # euclidean distance
    delta_rot_2 = mapper.normalize_angle(cur_pose[2]-trans_angle) # rotation from translation angle to final pose

    return delta_rot_1, delta_trans, delta_rot_"""
                        
# In world coordinates
def odom_motion_model(cur_pose, prev_pose, u):
    """ Odometry Motion Model

    Args:
        cur_pose  ([Pose]): Current Pose
        prev_pose ([Pose]): Previous Pose
        (rot1, trans, rot2) (float, float, float): A tuple with control data in the format 
                                                   format (rot1, trans, rot2) with units (degrees, meters, degrees)


    Returns:
        prob [float]: Probability p(x'|x, u)
    

    #This is an implementation of the Bayes filter following the Markov assumption on state x_t (cur_pose)
    #Transition probability/action model for the prediction step
    #It specifies how the robot state evolves over time as a function of robot controls u_t
    #p(x'|x,u) = (p(x,u | x') * p(x')) / (p(x,u)) = eta * p(x,u | x') * p(x')
    computed_control = compute_control(cur_pose, prev_pose);
    delta_trans = computed_control[0];
    delta_rot_1 = computed_control[1];
    delta_rot_2 = computed_control[2];
    
    #calculate probability of each condition 
    prob1 = loc.gaussian(delta_trans, u[0], loc.odom_trans_sigma);
    prob2 = loc.gaussian(delta_rot_1, u[1], loc.odom_rot_sigma);
    prob3 = loc.gaussian(delta_rot_2, u[2], loc.odom_rot_sigma);
    
    prob = prob1*prob2*prob3;
    
    #prob = loc.gaussian(computed_control[0], u[0], loc.odom_rot_sigma) * loc.gaussian(computed_control[1], u[1], loc.odom_trans_sigma) * loc.gaussian(computed_control[2], u[2], loc.odom_rot_sigma)
    return prob
    """
    # called Transition Probability / Action Model in lecture notes
    
    # find the movement between previous and current timestep
    movement = compute_control(cur_pose, prev_pose)
    
    # easier to understand implimentation:
    #P_rot_1 = loc.gaussian(movement[0], u[0], loc.odom_rot_sigma) # probability of rotation 1 given control
    #P_trans = loc.gaussian(movement[1], u[1], loc.odom_trans_sigma) # probability of translation given control
    #P_rot_2 = loc.gaussian(movement[2], u[2], loc.odom_rot_sigma) # probability of rotation 2 given control
    #prob = P_rot_1*P_trans*P_rot_2 # assume independent probabilities
    
    # faster implimentaion:
    prob = loc.gaussian(movement[0], u[0], loc.odom_rot_sigma) * loc.gaussian(movement[1], u[1], loc.odom_trans_sigma) * loc.gaussian(movement[2], u[2], loc.odom_rot_sigma)
    return prob

   

def prediction_step(cur_odom, prev_odom):
    """ Prediction step of the Bayes Filter.
    Update the probabilities in loc.bel_bar based on loc.bel from the previous time step and the odometry motion model.

    Args:
        cur_odom  ([Pose]): Current Pose
        prev_odom ([Pose]): Previous Pose
    """
    #MAX_CELLS_X: Maximum number of grid cells in the X direction.
    #MAX_CELLS_Y: Maximum number of grid cells in the Y direction.
    #MAX_CELLS_A: Maximum number of grid cells in the theta direction ('A' stands for 'Angle').
    #OBS_PER_CELL: Number of observations per cell. This is set to 18.
    #obs_views: A 4D Numpy array where the first three dimensions represents the cell index (cx, cy, ca) and the fourth dimension represents the precached true measurements (views)
    
    #this is equal to the control information of the robot

    u = compute_control(cur_odom, prev_odom) 
    #this is to find the probability of p(x'|x,u), state transition probability
    #prob = odom_motion_model(cur_odom, prev_odom, u) 
    #these loops are to step through every possible previous state (x,y.a)
    #this is because every state in "bel_bar" needs to be updated based on "bel" from the previous time step and the odometry motion model
    for x in range(0,loc.mapper.MAX_CELLS_X):
        for y in range(0,loc.mapper.MAX_CELLS_Y):
            for a in range(0,loc.mapper.MAX_CELLS_A):
                #this is equivalent to skipping a cell if the probability of its state is 0 (since 0.0001 is close to 0)
                if loc.bel[x, y, a] > 0.0001:
                    #these loops are to step through every possible previous state (x,y.a)
                    #this is because every state in "bel_bar" needs to be updated based on "bel" from the previous time step and the odometry motion model
                    for curr_x in range(0,loc.mapper.MAX_CELLS_X):
                        for curr_y in range(0,loc.mapper.MAX_CELLS_Y):
                            for curr_a in range(0,loc.mapper.MAX_CELLS_A):
                                loc.bel_bar[curr_x, curr_y, curr_a] = loc.bel_bar[curr_x, curr_y, curr_a] + odom_motion_model(loc.mapper.from_map(curr_x, curr_y, curr_a), mapper.from_map(x, y, a), u) * loc.bel[x, y, a]							

def sensor_model(obs, x_t):
    """ This is the equivalent of p(z|x).


    Args:
        obs ([ndarray]): A 1D array consisting of the measurements made in rotation loop
        x_t ([Pose]): State x of time t
        
    Returns:
        [ndarray]: Returns a 1D array of size 18 (=loc.OBS_PER_CELL) with the likelihood of each individual measurements
    """
    #Calculating measurement probability
    #It specifies how the measurements are generated from the robot state x_t
    #p(z|x) = p(x|z) * p(z) / p(x) = eta * p(x|z) * p(z)
                        
    prob_array = []
    #this is to loop over all 18 (OBS_PER_CELL) observations made by the robot 
    for i in range(0, loc.mapper.OBS_PER_CELL):
        #this is equal to the probability of finding the observation data in a normal Gaussian distibution gaussian(x, mu, sigma) f
        #this normal distribution is centered around the current state's views and has a standard deviation of the sensor_sigma
        prob_array.append(loc.gaussian(obs[i], loc.mapper.obs_views[x_t[0], x_t[1], x_t[2]], loc.sensor_sigma))
                        
    return prob_array

def update_step():
    """ Update step of the Bayes Filter.
    Update the probabilities in loc.bel based on loc.bel_bar and the sensor model.
    """
    #these loops are to step through every possible current state (x,y.a)
    #this is because every state in "bel" needs to be updated based on "bel_bar" and the sensor model from the current time step

    for x in range(0, mapper.MAX_CELLS_X):
        for y in range(0, mapper.MAX_CELLS_Y):
            for a in range(0, mapper.MAX_CELLS_A):
                #this is equivalent to skipping a cell if the probability of its state is 0 (since 0.0001 is close to 0)
                if loc.bel_bar[x, y, a] > 0.0001:
                    #this is to update the value of the current state's belief
                    #loc.bel[x, y, a] =  np.prod(sensor_model(loc.obs_range_data, (x, y, a))) * loc.bel_bar[x, y, a]
                    #this is to normalize the current state's belief (equivalent to multiplying by eta)
                    #loc.bel = loc.bel / np.sum(loc.bel)
                    loc.bel[x, y, a] = np.prod(loc.gaussian(loc.obs_range_data, mapper.obs_views[x, y, a, :], loc.sensor_sigma)) * loc.bel_bar[x, y, a]
                    loc.bel = loc.bel / np.sum(loc.bel) # normalize belief grid


# Run the Bayes Filter
The cell below utilizes the functions defined above to run each iteration of the Bayes filter algorithm to localize the robot in the grid. <br>

In each iteration:
- Execute robot motion (get $u_{t}$ as previous and current odom) 
- Perform prediction step (calculate $\overline{bel}$)
- Print information regarding Prediction step
- Execute robot rotation behavior to get observation data (get $z_{t}$)
- Perform update step (calculate $bel$)
- Print information regarding update step


**<ins>NOTE</ins>**: 
- During initial testing, you may want to limit the iteration to only the first time step (i.e t=0) instead of looping through the entire trajectory.
- <span style="color:rgb(0,150,0)">If you make changes to any of the functions above, make sure to re-run the above cell before executing the cell below.</span>
- The functions *print_prediction_stats()* and *print_update_stats()* are helper functions defined in robot_interface.py and may be changed to suit your needs.

In [6]:
# Reset Plots
loc.plotter.reset_plot()

# Intialize the Trajectory object
traj = Trajectory(loc)

# Run through each motion steps
for t in range(0, traj.total_time_steps):

    print("\n\n-----------------", t, "-----------------")
    
    prev_odom, current_odom, prev_gt, current_gt = traj.execute_time_step(t)
    
    # Prediction Step
    prediction_step(current_odom, prev_odom)
    loc.print_prediction_stats(plot_data=True)
    
    # Get Observation Data by executing a 360 degree rotation motion
    loc.get_observation_data()
    
    # Update Step
    update_step()
    loc.print_update_stats(plot_data=True)
        
    print("-------------------------------------")



----------------- 0 -----------------
 | Resetting Robot pose

---------- PREDICTION STATS -----------
GT index            :  (11, 9, 7)
Prior Bel index     :  (13, 9, 9) with prob =  0.5583522
POS ERROR      : (-0.311, 0.005, -32.345)
---------- PREDICTION STATS -----------
 | Executing Observation Loop at: 30 deg/s

---------- UPDATE STATS -----------
GT index      :  (11, 9, 7)
Bel index     :  (10, 10, 7) with prob =  0.9877671
Bel_bar prob at index =  0.00031077934799717064

GT     : (0.389, -0.095, -22.345)
Belief   : (0.100, 0.100, -30.000)
POS ERROR : (0.289, -0.195, 7.655)
---------- UPDATE STATS -----------
-------------------------------------


----------------- 1 -----------------

---------- PREDICTION STATS -----------
GT index            :  (13, 7, 8)
Prior Bel index     :  (10, 9, 9) with prob =  0.5950288
POS ERROR      : (0.572, -0.345, -15.157)
---------- PREDICTION STATS -----------
 | Executing Observation Loop at: 30 deg/s

---------- UPDATE STATS -----------
G